In [1]:
# do we do both together, or separately (probably do separately, and if different results, do together)
# in other words, do we compute the one factor that is the greatest determinant for both
# or do we compute the independently compute the greatest determining factors for each category
    # the factors may or may not be the same
# ask which factors they're talking about

In [2]:
# Approach:
# combine apprentice and journeymen datasets (at this point, I think we've used both together for everything, lol)
# label all the different factors (project, contractor, trade, craft level)
# use one-hot encoding (reason: label encoding [the alternative] works in a way 
    # which biases the influence of different features), and we don't want that
    # however, will likely need to address multicollinearity 
# resource: https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/
# use a linear regression/neural network to compute which feature has the highest coefficient/weight associated with it?

# alternatively, follow blog post: https://towardsdatascience.com/3-essential-ways-to-calculate-feature-importance-in-python-2f9149592155
# my suggested method is closest to method #1

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [4]:
df = pd.read_csv('WorkforceUtilizationReport-Journeymen.csv')
df = pd.concat([df, pd.read_csv('WorkforceUtilizationReport-Apprentice.csv')])

In [5]:
df.head()

,MONTH,YEAR,PROJECT,PROJECT_CODE,CONTRACTOR,CONSTRUCTION_TRADE,CRAFT_LEVEL,TOTAL_EMPLOYEE,CAUCASIAN,AFRICAN_AMERICAN,HISPANIC,ASIAN,NATIVE_AMERICAN,OTHER,NOT_SPECIFIED,TOTAL_FEMALE,TOTAL_MALE,HOURS_WORKED_PER_MONTH
0,8,2017,AEP1407E UT1 C AEP Utility Contract-Cape Cod,AEP 1407E UT1,Rise engineering,INSULATOR (PIPES & TANKS),Journeymen,16.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,16.0,HOURS_PER_MONTH
1,8,2017,CME0902 DC1 C Chief Medical Examiner Office We...,CME0902 DC1 CM,Crestview construction & trucking inc.,BACKHOE/FRONT-END LOADER,Journeymen,247.5,247.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,247.5,HOURS_PER_MONTH
2,8,2017,CME0902 DC1 C Chief Medical Examiner Office We...,CME0902 DC1 CM,Crestview construction & trucking inc.,DRIVER / GROUNDMAN CDL,Journeymen,127.0,127.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.0,HOURS_PER_MONTH
3,8,2017,CME0902 DC1 C Chief Medical Examiner Office We...,CME0902 DC1 CM,Crestview construction & trucking inc.,EQUIPMENT OPERATOR (Class B CDL),Journeymen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HOURS_PER_MONTH
4,8,2017,CME0902 DC1 C Chief Medical Examiner Office We...,CME0902 DC1 CM,Crestview construction & trucking inc.,LABORER,Journeymen,61.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,HOURS_PER_MONTH


In [6]:
df.columns

Index(['MONTH', 'YEAR', 'PROJECT', 'PROJECT_CODE', 'CONTRACTOR',
       'CONSTRUCTION_TRADE', 'CRAFT_LEVEL', 'TOTAL_EMPLOYEE', 'CAUCASIAN',
       'AFRICAN_AMERICAN', 'HISPANIC', 'ASIAN', 'NATIVE_AMERICAN', 'OTHER',
       'NOT_SPECIFIED', 'TOTAL_FEMALE', 'TOTAL_MALE',
       'HOURS_WORKED_PER_MONTH'],
      dtype='object')

In [7]:
poc_cols = ['AFRICAN_AMERICAN', 'HISPANIC', 'ASIAN', 'NATIVE_AMERICAN', 'OTHER']
for i in range(df.shape[0]):
    sum = 0
    for col in poc_cols:
        sum += df.iloc[i][col]
    print(str(i) + ' ' + str(sum))
    df.iloc[i]['POC'] = sum

0 16.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 58.0
9 12.0
10 26.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 20.0
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 368.0
24 194.0
25 253.0
26 0.0
27 0.0
28 0.0
29 0.0
30 6.0
31 0.0
32 800.0
33 248.0
34 32.0
35 0.0
36 0.0
37 23.0
38 57.0
39 0.0
40 0.0
41 0.0
42 0.0
43 0.0
44 1320.0
45 0.0
46 0.0
47 0.0
48 0.0
49 0.0
50 72.0
51 0.0
52 0.0
53 0.0
54 0.0
55 0.0
56 0.0
57 0.0
58 0.0
59 0.0
60 0.0
61 0.0
62 0.0
63 160.0
64 0.0
65 0.0
66 0.0
67 0.0
68 25.5
69 0.0
70 8.0
71 0.0
72 0.0
73 0.0
74 37.5
75 3.0
76 0.0
77 0.0
78 0.0
79 0.0
80 0.0
81 0.0
82 0.0
83 0.0
84 0.0
85 8.0
86 0.0
87 0.0
88 0.0
89 0.0
90 8.0

C:\Users\ASUS\anaconda3\envs\dcamm_env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\ASUS\anaconda3\envs\dcamm_env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)



91 0.0
92 0.0
93 0.0
94 0.0
95 0.0
96 253.5
97 186.0
98 334.0
99 0.0
100 0.0
101 0.0
102 40.0
103 0.0
104 4.0
105 0.0
106 0.0
107 0.0
108 0.0
109 0.0
110 0.0
111 0.0
112 40.0
113 624.0
114 0.0
115 0.0
116 0.0
117 0.0
118 0.0
119 0.0
120 1712.0
121 0.0
122 0.0
123 0.0
124 0.0
125 578.0
126 112.0
127 0.0
128 0.0
129 72.0
130 0.0
131 0.0
132 2.0
133 92.0
134 0.0
135 0.0
136 0.0
137 0.0
138 0.0
139 0.0
140 0.0
141 0.0
142 0.0
143 0.0
144 0.0
145 0.0
146 0.0
147 0.0
148 168.0
149 0.0
150 0.0
151 0.0
152 0.0
153 0.0
154 13.0
155 0.0
156 0.0
157 0.0
158 24.0
159 0.0
160 0.0
161 0.0
162 0.0
163 0.0
164 0.0
165 0.0
166 0.0
167 0.0
168 0.0
169 0.0
170 0.0
171 53.0
172 0.0
173 0.0
174 0.0
175 176.0
176 0.0
177 72.0
178 0.0
179 0.0
180 108.0
181 5.0
182 116.0
183 0.0
184 0.0
185 140.0
186 20.0
187 0.0
188 0.0
189 16.0
190 0.0
191 0.0
192 1.5
193 0.0
194 0.0
195 0.0
196 8.0
197 128.0
198 0.0
199 0.0
200 0.0
201 5.0
202 0.0
203 0.0
204 0.0
205 0.0
206 0.0
207 0.0
208 0.0
209 2.0
210 0.0
211 18.0
21

In [8]:
to_transform = ['PROJECT_CODE', 'CONTRACTOR', 'TOTAL_EMPLOYEE', 'CONSTRUCTION_TRADE', 'CRAFT_LEVEL']
df_women = df.drop(['MONTH', 'YEAR', 'PROJECT', 'CAUCASIAN',
       'AFRICAN_AMERICAN', 'HISPANIC', 'ASIAN', 'NATIVE_AMERICAN', 'OTHER',
       'NOT_SPECIFIED', 'TOTAL_MALE', 'HOURS_WORKED_PER_MONTH'], axis=1)
df_poc = df.drop(['MONTH', 'YEAR', 'PROJECT', 'CAUCASIAN',
       'AFRICAN_AMERICAN', 'HISPANIC', 'ASIAN', 'NATIVE_AMERICAN', 'OTHER',
       'NOT_SPECIFIED', 'TOTAL_MALE', 'HOURS_WORKED_PER_MONTH'], axis=1)
final_cols = ['PROJECT_CODE', 'CONTRACTOR', 'TOTAL_EMPLOYEE', 'CONSTRUCTION_TRADE', 'CRAFT_LEVEL', 'TOTAL_FEMALE', 'POC']

In [9]:
df_poc.columns

Index(['PROJECT_CODE', 'CONTRACTOR', 'CONSTRUCTION_TRADE', 'CRAFT_LEVEL',
       'TOTAL_EMPLOYEE', 'TOTAL_FEMALE'],
      dtype='object')

In [10]:
df_women.columns

Index(['PROJECT_CODE', 'CONTRACTOR', 'CONSTRUCTION_TRADE', 'CRAFT_LEVEL',
       'TOTAL_EMPLOYEE', 'TOTAL_FEMALE'],
      dtype='object')

In [11]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_poc = pd.DataFrame(OH_encoder.fit_transform(df_poc[to_transform]))
OH_cols_women = pd.DataFrame(OH_encoder.fit_transform(df_women[to_transform]))

# One-hot encoding removed index; put it back
OH_cols_poc.index = df_poc.index
OH_cols_women.index = df_women.index

df_poc = df_poc.drop(to_transform, axis=1)
df_women = df_women.drop(to_transform, axis=1)

final_poc = pd.concat([df_poc, OH_cols_poc], axis=1)
final_women = pd.concat([df_women, OH_cols_women], axis=1)

In [13]:
final_poc

,TOTAL_FEMALE,0,1,2,3,4,5,6,7,8,...,2164,2165,2166,2167,2168,2169,2170,2171,2172,2173
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
